In [48]:
import re

latex_input = r"""
\[
       \begin{aligned}
       &s_{2,3}-s_{1,4}-s_{3,4}, \\
       &s_{1,4}s_{2,4}-s_{2,2}s_{3,4}+s_{1,4}s_{4,4}+s_{3,4}s_{4,4}, \\
       &s_{3,3}s_{2,4}-s_{1,4}s_{3,4}-s_{3,4}^{2}, \\
       &s_{1,3}s_{2,4}-s_{1,2}s_{3,4}-s_{1,4}s_{3,4}+s_{1,3}s_{4,4}, \\
       &s_{1,4}^{2}-s_{1,2}s_{3,4}+s_{1,3}s_{4,4}, \\
       &s_{3,3}s_{1,4}-s_{1,3}s_{3,4}, \\
       &s_{2,2}s_{1,4}-s_{1,2}s_{2,4}, \\
       &s_{1,2}s_{1,4}-s_{1,1}s_{2,4}-s_{1,4}s_{3,4}+s_{1,3}s_{4,4}, \\
       &s_{2,2}s_{3,3}-s_{1,2}s_{3,4}-s_{1,4}s_{3,4}-s_{3,3}s_{4,4}, \\
       &s_{1,2}s_{3,3}-s_{1,1}s_{3,4}-s_{1,3}s_{3,4}, \\
       &s_{2,2}s_{1,3}-s_{1,1}s_{2,4}-s_{1,2}s_{3,4}-s_{1,4}s_{3,4}+s_{1,3}s_{4,4}, \\
       &s_{1,2}s_{1,3}-s_{1,1}s_{1,4}-s_{1,3}s_{1,4}, \\
       &s_{1,2}^{2}-s_{1,1}s_{2,2}-s_{1,2}s_{3,4}+s_{1,1}s_{4,4}+s_{1,3}s_{4,4}, \\
       &s_{1,3}s_{1,4}s_{3,4}-s_{1,1}s_{3,4}^{2}-s_{1,3}s_{3,4}^{2}+s_{1,3}s_{3,3}s_{4,4}, \\
       &s_{1,2}s_{2,4}^{2}-s_{2,2}^{2}s_{3,4}+s_{1,2}s_{2,4}s_{4,4}+s_{2,2}s_{3,4}s_{4,4}, \\
       &s_{1,1}s_{2,4}^{2}-s_{1,2}s_{2,2}s_{3,4}+s_{2,2}s_{3,4}^{2}+s_{1,1}s_{2,4}s_{4,4}-s_{1,4}s_{3,4}s_{4,4}-s_{3,4}^{2}s_{4,4}, \\
       &s_{1,3}^{2}s_{3,4}^{2}-s_{1,1}s_{3,3}s_{3,4}^{2}-s_{1,3}s_{3,3}s_{3,4}^{2}+s_{1,3}s_{3,3}^{2}s_{4,4}.
       \end{aligned}
       \]
    
"""

def make_sage_code(latex):
    # Match ideal or aligned environment
    ideal_match = re.search(r'\\texttt\{ideal\}\{\}\\left\((.*?)\\right\)', latex, re.DOTALL)
    aligned_match = re.search(r'\\begin\{aligned\}(.*?)\\end\{aligned\}', latex, re.DOTALL)
    if ideal_match:
        expr = ideal_match.group(1)
    elif aligned_match:
        expr = aligned_match.group(1)
    else:
        raise ValueError("No supported LaTeX ideal format found.")
    # Normalize LaTeX syntax
    expr = expr.replace(r'\,', '*')
    expr = expr.replace(r'\\', ' ')
    expr = expr.replace('&', '')
    expr = re.sub(r'%.*', '', expr)
    expr = re.sub(r'\s+', ' ', expr)
    # Convert variables s_{i,j} to xij
    expr = re.sub(r's\s*_\{\s*(\d)\s*,\s*(\d)\s*\}', lambda m: f"x{m.group(1)}{m.group(2)}", expr)
    # Replace powers: ^{2} → **2
    expr = re.sub(r'\^\s*\{\s*(\d+)\s*\}', r'**\1', expr)
    # Insert explicit multiplications
    expr = re.sub(r'(?<=[\d\)])(?=x\d{2})', '*', expr)  # between number or ) and variable
    expr = re.sub(r'(?<=x\d{2})(?=x\d{2})', '*', expr)  # between variables
    # Split polynomials
    polys = [p.strip() for p in re.split(r',(?![^()]*\))', expr) if p.strip()]
    # Remove any trailing periods from the last polynomial (safe fix)
    polys[-1] = polys[-1].rstrip('.')
    # Extract variables
    all_vars = sorted(set(re.findall(r'x\d{2}', ' '.join(polys))))
    ring = f"R.<{', '.join(all_vars)}> = PolynomialRing(QQ)"
    poly_defs = [f"P{i+1} = {p}" for i, p in enumerate(polys)]
    ideal = f"I = R.ideal([{', '.join(f'P{i+1}' for i in range(len(polys)))}])"
    return "\n".join([ring] + poly_defs + [ideal, "gbT(I)"])
print(make_sage_code(latex_input))

R.<x11, x12, x13, x14, x22, x23, x24, x33, x34, x44> = PolynomialRing(QQ)
P1 = x23-x14-x34
P2 = x14*x24-x22*x34+x14*x44+x34*x44
P3 = x33*x24-x14*x34-x34**2
P4 = x13*x24-x12*x34-x14*x34+x13*x44
P5 = x14**2-x12*x34+x13*x44
P6 = x33*x14-x13*x34
P7 = x22*x14-x12*x24
P8 = x12*x14-x11*x24-x14*x34+x13*x44
P9 = x22*x33-x12*x34-x14*x34-x33*x44
P10 = x12*x33-x11*x34-x13*x34
P11 = x22*x13-x11*x24-x12*x34-x14*x34+x13*x44
P12 = x12*x13-x11*x14-x13*x14
P13 = x12**2-x11*x22-x12*x34+x11*x44+x13*x44
P14 = x13*x14*x34-x11*x34**2-x13*x34**2+x13*x33*x44
P15 = x12*x24**2-x22**2*x34+x12*x24*x44+x22*x34*x44
P16 = x11*x24**2-x12*x22*x34+x22*x34**2+x11*x24*x44-x14*x34*x44-x34**2*x44
P17 = x13**2*x34**2-x11*x33*x34**2-x13*x33*x34**2+x13*x33**2*x44
I = R.ideal([P1, P2, P3, P4, P5, P6, P7, P8, P9, P10, P11, P12, P13, P14, P15, P16, P17])
gbT(I)
